**Dependent demand constraint exercise**

You are developing a production plan for 5 products (A through E). One unit of product E comprises 2 units of A, and 1 unit of C. Product C is also sold directly to customers. Your planning horizon is 3 months. You are looking to determine how much of each product to produce by month that meets the customer's demand and minimizes the total costs.

A Pandas DataFrame named `demand` is provided, which contains the monthly customer demand for each product.

In [2]:
from pulp import *
import pandas as pd

In [3]:
dmnd = {'A': [0, 0, 0], 'B': [1000, 700, 600], 'C': [300, 200, 0], 'D': [300, 0, 0], 'E': [800, 400, 500]}
demand = pd.DataFrame(dmnd, index=[0, 1, 2])
demand

,A,B,C,D,E
0,0,1000,300,300,800
1,0,700,200,0,400
2,0,600,0,0,500


In [ ]:
# Initialize, Define Decision Vars., Objective Function, and Constraints
model = LpProblem("Aggregate_Production_Planning", LpMinimize)
time = [0, 1, 2]
prod = ['A', 'B', 'C', 'D', 'E']
X = LpVariable.dicts("prod", [(t, p) for p in prod for t in time], 
                     lowBound=0, cat="Integer")
model += lpSum([costs.loc[t, p]*X[(t, p)] for p in prod for t in time])

# meeting demands
for p in prod:
    for t in time:
        model += X[(t, p)] >= demand.loc[t, p]
        
# Define Dependent Demand Constraints
for t in time:
    model += 2 * X[(t, 'E')] <= X[(t, 'A')]
    model += 1 * X[(t, 'E')] + demand.loc[t, 'C'] <= X[(t, 'C')]